In [1]:
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0

In [7]:
!pip install -q faiss-cpu
!pip install -q git+https://github.com/openai/CLIP.git

In [4]:
!pip install -q translate
!pip install -q underthesea==1.3.5a3
!pip install -q underthesea[deep]
!pip install -q pyvi
!pip install -q langdetect
!pip install -q googletrans==3.1.0a0
!pip install -q peft
!pip install bitsandbytes
!pip install transformers
!pip install flash-attn
!pip install -U sentence-transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
jupyterlab 4.2.3 requires httpx>=0.25.0, but you have httpx 0.13.3 which is incompatible.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 44.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_at

In [8]:
import os
import torch
import clip
from PIL import Image
import faiss
import numpy as np
import json
import matplotlib.pyplot as plt
import math
import googletrans
import translate
import glob
import underthesea
import sys
import time
from tqdm import tqdm
from pyvi import ViUtils, ViTokenizer
from difflib import SequenceMatcher
from langdetect import detect
from pathlib import Path
import re

In [9]:
ROOT = Path(os.getcwd()).resolve()

# Add ROOT to sys.path
sys.path.append(str(ROOT))

# Determine the working directory
if len(ROOT.parents) > 1:
    WORK_DIR = ROOT.parents[0]
else:
    WORK_DIR = ROOT  # Fallback to ROOT if it doesn't have enough parents

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model, preprocess = clip.load("ViT-B/32", device=device)

# paths = f"{WORK_DIR}/data/keyframes"
# des_path =  f"{WORK_DIR}/working//dicts/npy_vlm"
# os.makedirs(des_path, exist_ok=True)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 139MiB/s]


In [10]:
from transformers import DPRContextEncoder, AutoProcessor, DPRContextEncoderTokenizer, BlipModel,TrOCRProcessor, VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer, BitsAndBytesConfig, BlipForConditionalGeneration
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from sentence_transformers import SentenceTransformer


# tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base', use_fast=False)
# embedding_model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base').to(device)

embedding_model= SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

2024-07-24 04:53:59.209356: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 04:53:59.209457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 04:53:59.369475: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
#WORK_DIR =

paths = f"{WORK_DIR}/data/keyframes"
# Define paths

des_path =  f"{WORK_DIR}/data/dicts/npy_vlm"
os.makedirs(des_path, exist_ok=True)

In [14]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [15]:
from transformers import pipeline

model_id = "llava-hf/llava-1.5-7b-hf"

pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

In [17]:

def vlm_image(image_path):
  image = Image.open(image_path)
  max_new_tokens = 200
  prompt = "USER: <image>\nWhat are the things I should be cautious about when I visit this place?\nASSISTANT:"

  outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
  return outputs[0]["generated_text"]

  return processor.decode(output[0], skip_special_tokens=True)



In [18]:

for keyframe in tqdm(os.listdir(paths)):
  path_keyframe = os.path.join(paths,keyframe)
  video_paths = sorted(glob.glob(f"{path_keyframe}/*/"))
  video_paths = ['/'.join(i.split('/')[:-1]) for i in video_paths]

  start_time = time.time()
  for vd_path in video_paths:

    re_feats = []
    keyframe_paths = glob.glob(f'{vd_path}/*.jpg')
    keyframe_paths = sorted(keyframe_paths, key=lambda x : x.split('/')[-1].replace('.jpg',''))

    for keyframe_path in tqdm(keyframe_paths):


      #text = ocr_image(keyframe_path)

      #//////////////////////////////////
      text = vlm_image(keyframe_path)
      #//////////////////////////////////
      if detect(text) == 'vi' :
        text = Translation(text)

      # Convert text to embedding vector
      #embedding = embedding_model(**tokenizer(text, return_tensors="pt", padding=True, truncation=True,max_length=512, add_special_tokens = True)).pooler_output.detach().numpy()
      embeddings = embedding_model.encode(text)
      # Append embedding to re_feats list
      re_feats.append(embeddings)

    name_npy = vd_path.split('/')[-1]

    # Construct output file path
    outfile = os.path.join(des_path, f'{name_npy}.npy')

    # Ensure the directory exists before saving
    os.makedirs(des_path, exist_ok=True)
    np.save(outfile, re_feats)

    print(f"Processed {vd_path} in {time.time() - start_time} seconds")


  0%|          | 0/244 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 1/244 [00:11<44:39, 11.02s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  1%|          | 2/244 [00:22<45:14, 11.22s/it]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|█████████▉| 243/244 [34:20<00:09,  9.15s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 244/244 [34:28<00:00,  8.48s/it]


NameError: name 'start_time' is not defined

In [ ]:
feature_shape = 512


def write_bin_file_ocr(bin_path: str, npy_path: str, method='cosine'):
    if method in 'L2':
      index = faiss.IndexFlatL2(feature_shape)
    elif method in 'cosine':
      index = faiss.IndexFlatIP(feature_shape)
    else:
      assert f"{method} not supported"
    npy_files = glob.glob(os.path.join(npy_path, "*.npy"))
    npy_files_sorted = sorted(npy_files)

    for npy_file in npy_files_sorted:
        feats = np.load(npy_file)
        print(f"Loaded {npy_file}, shape: {feats.shape}")


        # Convert to float32 and reshape to match feature_shape
        feats = feats.astype(np.float32)
        feats = feats.reshape(-1, feats.shape[-1])

        # Resize or trim feats_normalized to match feature_shape if necessary
        if feats.shape[1] != feature_shape:
            feats = feats[:, :feature_shape]

        assert feats.shape[1] == feature_shape, \
            f"Query features dimension {feats.shape[1]} do not match index dimension {feature_shape}"

        # Add to Faiss index
        index.add(feats)

    # Write the Faiss index to disk
    faiss.write_index(index, os.path.join(bin_path, f"faiss_VLM_{method}.bin"))
    print(f'Saved {os.path.join(bin_path, f"faiss_VLM_{method}.bin")}')


# write ocr
write_bin_file_ocr(bin_path=f"{WORK_DIR}/data/dicts/bin_vlm", npy_path=f"{WORK_DIR}/data/dicts/npy_vlm")



In [20]:
re_feats

[array([ 6.96230084e-02, -4.76951301e-02,  2.96242610e-02, -4.80196439e-03,
         4.08199467e-02,  9.60981008e-03,  1.09238394e-01, -4.61272821e-02,
        -2.90390830e-02, -7.49663562e-02,  1.08216209e-02, -2.64952201e-02,
        -8.02921131e-03, -3.34884971e-03, -5.50004691e-02, -1.24515928e-01,
         1.44051000e-01, -6.76815957e-02, -2.56896894e-02,  4.67843860e-02,
         2.49097440e-02, -1.03631495e-02, -2.03340854e-02,  3.30182947e-02,
        -3.32783721e-02,  7.15212803e-03,  5.63152805e-02,  4.31651585e-02,
        -2.11944617e-02, -5.03317229e-02, -1.60983801e-02, -2.54912283e-02,
         8.58755335e-02, -1.12230228e-02, -2.23396160e-03,  4.56140600e-02,
         7.49859959e-02, -1.52744493e-02,  4.54771109e-02,  4.72492613e-02,
        -5.13655916e-02, -8.55976194e-02,  2.27248371e-02, -1.50348516e-02,
        -8.09317380e-02, -9.35754701e-02,  2.33401656e-02, -1.39263228e-01,
         4.30909917e-03, -7.17124194e-02, -2.46276148e-02, -3.46071534e-02,
         2.6

In [24]:
des_path =  f"kaggle/working/"
name_npy = vd_path.split('/')[-1]

# Construct output file path
outfile = os.path.join(des_path, f'{name_npy}.npy')

# Ensure the directory exists before saving
os.makedirs(des_path, exist_ok=True)
np.save(outfile, re_feats)